In [1]:
%load_ext autoreload
%autoreload 2

# Import libs

In [2]:
import os

import pandas as pd
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


# Reading data

In [3]:
path = "../"
train_data = pd.read_csv(os.path.join(path, "train.csv"))
# val_data = pd.read_csv(os.path.join('../', path, "test-with-targets.csv"))  # TODO change back
test_data = pd.read_csv(os.path.join(path, "test.csv"))
print(f"Number of rows and columns in the train data set: {train_data.shape}")
print(f"Number of rows and columns in the test data set: {test_data.shape}")


Number of rows and columns in the train data set: (5495, 4)
Number of rows and columns in the test data set: (788, 3)


In [4]:
train_data.head()

,id,movie_name,movie_description,target
0,3525e31d,Hellraiser,A new take on Clive Barker's 1987 horror class...,Horror
1,051f6309,Hocus Pocus 2,It's been 29 years since someone lit the Black...,Kids
2,12a9bfcf,X,"In 1979, a group of young filmmakers set out t...",Horror
3,e5373c77,Piggy,With the summer sun beating down on her rural ...,Horror
4,473cdb82,Deadstream,After a public controversy left him disgraced ...,Horror


In [5]:
train_data.groupby("target").describe()

id                       movie_name                                 \
        count unique       top freq      count unique                     top   
target                                                                          
Action    704    704  68cfcb79    1        704    697  Assault on Precinct 13   
Comedy   1269   1269  f00aac8c    1       1269   1264     Father of the Bride   
Drama    1202   1202  6569d7af    1       1202   1196            Little Women   
Horror    974    974  3525e31d    1        974    942                  Carrie   
Kids      552    552  051f6309    1        552    539               Pinocchio   
Mystery   794    794  e5da8af4    1        794    769                   Alone   

             movie_description         \
        freq             count unique   
target                                  
Action     2               704    703   
Comedy     2              1269   1269   
Drama      2              1202   1202   
Horror     3               974    974   
Kids       3               552    552   
Mystery    3               794    793   

                                                                 
                                                       top freq  
target                                                           
Action   Wisecracking mercenary Deadpool meets Russell,...    2  
Comedy   To show support for his neighborhood friends s...    1  
Drama    Vince used to be in the hottest boy band aroun...    1  
Horror   A new take on Clive Barker's 1987 horror class...    1  
Kids     It's been 29 years since someone lit the Black...    1  
Mystery  A mysterious figure stalks a pop star when she...    2

In [6]:
le = LabelEncoder()  # for label encoding!

y_train = le.fit_transform(train_data["target"])
# y_val = le.transform(val_data["target"])

# Preparing the data and creating Catboost model

In [7]:
X_train = train_data["movie_description"]
# X_val = val_data["movie_description"]
X_test = test_data['movie_description']
X_train.head()

0    A new take on Clive Barker's 1987 horror class...
1    It's been 29 years since someone lit the Black...
2    In 1979, a group of young filmmakers set out t...
3    With the summer sun beating down on her rural ...
4    After a public controversy left him disgraced ...
Name: movie_description, dtype: object

In [8]:
model = CatBoostClassifier(
    iterations=200,
    depth=2,
    random_seed=1
)

model.fit(
    X_train,
    y_train,
    text_features=[0],
    verbose=False
)

### Pool is an internal data structure that is used by CatBoost

In [9]:
# cv_dataset_val = Pool(
#     data=X_val,
#     text_features=[0]
# )

In [10]:
# predictions = model.predict(cv_dataset_val)

In [11]:
# accuracy_score(y_val, predictions)

In [12]:
# Preparing data in Pool format
cv_dataset_test = Pool(
    data=X_test,
    text_features=[0]
)
predict_scores = model.predict(cv_dataset_test)
predictions = le.inverse_transform(predict_scores)

/home/limp/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sample_submission.head()

,id,target
0,d996f823,Kids
1,1cf01f9c,Kids
2,856ea05c,Kids
3,c97899ee,Kids
4,73f0740f,Kids


In [14]:
sample_submission["target"] = predictions
sample_submission.head()

,id,target
0,d996f823,Kids
1,1cf01f9c,Drama
2,856ea05c,Mystery
3,c97899ee,Horror
4,73f0740f,Comedy


In [15]:
sample_submission.to_csv(os.path.join(path, "submission.csv"), index=False)

In [16]:
# Accuracy 0.6002